# Using grid index systems in Mosaic

In [0]:
from pyspark.sql.functions import *
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

Set operations over big geospatial datasets become very expensive without some form of spatial indexing.

Spatial indexes not only allow operations like point-in-polygon joins to be partitioned but, if only approximate results are required, can be used to reduce these to deterministic SQL joins directly on the indexes.

![example h3 point-in-poly image](https://databricks.com/wp-content/uploads/2021/01/blog-geospatial-3.jpg)

The workflow for a point-in-poly spatial join might look like the following:

## 1. Read the source point and polygon datasets.

In [0]:
drop_cols = [
  "rate_code_id", "store_and_fwd_flag", "dropoff_longitude",
  "dropoff_latitude", "payment_type", "fare_amount",
  "extra", "mta_tax", "tip_amount", "tolls_amount",
  "total_amount"
]

trips = (
  spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")
  .drop(*drop_cols)
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
)

trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+
 VTS|2010-02-27 00:58:00|2010-02-27 01:07:00| 1| 1.53| -73.993507| 40.733387|
 VTS|2009-05-14 18:37:00|2009-05-14 18:45:00| 1| 2.4| -73.991153| 40.72974|
 VTS|2009-10-11 00:48:00|2009-10-11 01:05:00| 1| 4.21| -74.005193| 40.721045|
 VTS|2010-02-19 12:59:00|2010-02-19 13:06:00| 4| 1.08| -74.010147| 40.72597|
 CMT|2010-05-11 12:45:26|2010-05-11 13:04:11| 1| 3.7| -74.009204| 40.713216|
 VTS|2009-12-16 11:03:00|2009-12-16 11:12:00| 1| 2.18| -73.999928| 40.721773|
 VTS|2010-02-18 07:45:00|2010-02-18 07:51:00| 1| 0.77| -74.017235| 40.715668|
 CMT|2010-09-17 22:27:28|2010-09-17 22:33:06| 1| 1.8| -74.231578| 40.583343|
 VTS|2009-05-11 20:59:00|2009-05-11 21:06:00| 1| 1.38| -73.992268| 40.728883|
 CMT|2010-06-03 06:43:18|2010-06-03 06:52:02| 1| 2.8| -74.011603| 40.706008|
 CMT|2009-05-30 22:50:38|2009-05-30 23:18:24| 1| 4.0| -74.008459| 40.731229|
 VTS|2010-02-24 17:22:00|2010-02-24 17:31:00| 1| 2.71| -74.011938| 40.701655|
 CMT|2009-12-15 20:10:59|2009-12-15 20:24:43| 1| 1.8| -73.99516| 40.73184|
 VTS|2010-06-21 21:02:00|2010-06-21 21:38:00| 1| 19.91| -74.010312| 40.70506|
 CMT|2010-02-18 21:02:46|2010-02-18 21:10:11| 1| 1.3| -73.989271| 40.726147|
 CMT|2009-12-03 18:51:31|2009-12-03 18:55:44| 1| 0.9| -73.993241| 40.722358|
 VTS|2009-10-11 19:17:00|2009-10-11 19:26:00| 1| 1.59| -74.007627| 40.725195|
 DDS|2010-05-09 11:42:02|2010-05-09 12:05:29| 1| 11.6| -74.009373| 40.71052|
 CMT|2009-05-08 11:31:52|2009-05-08 11:41:24| 1| 0.9| -73.996071| 40.732786|
 VTS|2009-04-04 22:28:00|2009-04-04 22:36:00| 1| 1.3| -73.998395| 40.71833|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+
only showing top 20 rows

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .repartition(sc.defaultParallelism)
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 Brooklyn| 255| 255|Williamsburg (Nor...|{6, [[[-73.961760...|
 Bronx| 147| 147| Longwood|{6, [[[-73.895152...|
 Queens| 203| 203| Rosedale|{6, [[[-73.727535...|
 Bronx| 182| 182| Parkchester|{6, [[[-73.854651...|
 Queens| 124| 124| Howard Beach|{6, [[[-73.850964...|
 Queens| 101| 101| Glen Oaks|{6, [[[-73.701347...|
 Bronx| 81| 81| Eastchester|{6, [[[-73.823668...|
 Brooklyn| 178| 178| Ocean Parkway South|{6, [[[-73.970804...|
 Brooklyn| 72| 72|East Flatbush/Rem...|{6, [[[-73.920213...|
 Manhattan| 249| 249| West Village|{6, [[[-74.002506...|
 Queens| 180| 180| Ozone Park|{6, [[[-73.841196...|
 Manhattan| 127| 127| Inwood|{6, [[[-73.911224...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Bronx| 60| 60| Crotona Park East|{6, [[[-73.878238...|
 Queens| 117| 117| Hammels/Arverne|{6, [[[-73.776358...|
Staten Island| 84| 84|Eltingville/Annad...|{6, [[[-74.160057...|
 Manhattan| 153| 153| Marble Hill|{6, [[[-73.906665...|
 Queens| 201| 201| Rockaway Park|{6, [[[-73.818499...|
 Queens| 129| 129| Jackson Heights|{6, [[[-73.875865...|
 Manhattan| 202| 202| Roosevelt Island|{6, [[[-73.941800...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

## 2. Compute the resolution of index required to optimize the join.

In [0]:
from mosaic import MosaicAnalyzer

analyzer = MosaicAnalyzer()
optimal_resolution = analyzer.get_optimal_resolution(geoJsonDF, "geometry")
optimal_resolution

Out[4]: 9

## 3. Apply the index to the set of points in your left-hand dataframe.
This will generate an index value that corresponds to the grid ‘cell’ that this point occupies.

In [0]:
from mosaic import point_index
indexed_trips = trips.withColumn("ix", point_index(lng="pickup_longitude", lat="pickup_latitude", resolution=lit(optimal_resolution)))
indexed_trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude| ix|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------------+
 DDS|2009-01-17 18:49:57|2009-01-17 18:56:29| 3| 1.2| -74.004043| 40.733409|617733151092113407|
 VTS|2009-10-16 22:02:00|2009-10-16 22:08:00| 1| 1.1| -74.010903| 40.71624|617733151084773375|
 CMT|2009-11-08 22:20:44|2009-11-08 22:31:23| 1| 1.9| -74.003029| 40.733385|617733151092113407|
 VTS|2009-01-08 18:13:00|2009-01-08 18:33:00| 2| 4.18| -74.0079| 40.712012|617733151021334527|
 CMT|2009-11-18 21:50:12|2009-11-18 22:05:19| 1| 5.8| -73.992515| 40.694106|617733151038111743|
 VTS|2009-12-28 20:34:00|2009-12-28 20:43:00| 5| 1.95| -74.003597| 40.732455|617733151092113407|
 CMT|2009-11-13 11:09:32|2009-11-13 11:26:23| 1| 2.5| -74.003922| 40.725683|617733151080054783|
 VTS|2009-02-10 08:47:00|2009-02-10 08:52:00| 5| 1.21| -74.006937| 40.703955|617733151023431679|
 VTS|2009-12-03 09:39:00|2009-12-03 09:51:00| 1| 4.05| -73.99822| 40.723497|617733151079792639|
 VTS|2009-12-09 09:49:00|2009-12-09 09:56:00| 5| 0.68| -74.017367| 40.704827|617733151011373055|
 CMT|2009-09-17 20:24:03|2009-09-17 20:30:15| 2| 0.9| -74.005168| 40.733361|617733151092637695|
 VTS|2010-10-25 17:51:00|2010-10-25 18:08:00| 1| 5.04| -74.009903| 40.70333|617733151012683775|
 CMT|2009-09-14 20:57:01|2009-09-14 21:17:40| 2| 3.5| -73.99733| 40.72073|617733151089491967|
 CMT|2009-05-13 11:56:35|2009-05-13 12:18:13| 1| 2.8| -73.992159| 40.731001|617733151087919103|
 VTS|2009-09-19 23:04:00|2009-09-19 23:11:00| 5| 1.32| -73.993007| 40.72802|617733151086870527|
 DDS|2009-11-11 20:18:05|2009-11-11 20:26:22| 3| 2.1| -74.013306| 40.707776|617733151012421631|
 CMT|2009-02-23 15:39:20|2009-02-23 15:41:53| 1| 0.2| -74.00337| 40.735527|617733150904680447|
 VTS|2009-05-23 13:13:00|2009-05-23 13:29:00| 1| 4.61| -74.017047| 40.7054|617733151011373055|
 CMT|2009-01-07 20:34:48|2009-01-07 20:46:21| 1| 2.1| -74.007879| 40.730181|617733151091589119|
 CMT|2009-10-01 22:24:51|2009-10-01 22:30:59| 2| 0.4| -73.990715| 40.724219|617733151090016255|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------------+
only showing top 20 rows

## 4. Compute the set of indices that fully covers each polygon in the right-hand dataframe
This is commonly referred to as a mosaic_polyfill operation.

In [0]:
from mosaic import mosaic_polyfill

indexed_neighbourhoods = (
  geoJsonDF
  .select("*", mosaic_polyfill("geometry", lit(optimal_resolution)).alias("ix_set"))
  .drop("geometry")
)

indexed_neighbourhoods.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| ix_set|
+-------------+-----------+--------+--------------------+--------------------+
 Brooklyn| 255| 255|Williamsburg (Nor...|[6177331238270402...|
 Bronx| 147| 147| Longwood|[6177331231197757...|
 Queens| 203| 203| Rosedale|[6177331218140364...|
 Bronx| 182| 182| Parkchester|[6177331207798783...|
 Queens| 124| 124| Howard Beach|[6177331236427530...|
 Queens| 101| 101| Glen Oaks|[6177331217099653...|
 Bronx| 81| 81| Eastchester|[6177331206343884...|
 Brooklyn| 178| 178| Ocean Parkway South|[6177331514952908...|
 Brooklyn| 72| 72|East Flatbush/Rem...|[6177331520232488...|
 Manhattan| 249| 249| West Village|[6177331509046804...|
 Queens| 180| 180| Ozone Park|[6177331236393451...|
 Manhattan| 127| 127| Inwood|[6177331229842472...|
 Queens| 9| 9| Auburndale|[6177331216591093...|
 Bronx| 60| 60| Crotona Park East|[6177331231871467...|
 Queens| 117| 117| Hammels/Arverne|[6177331353191710...|
Staten Island| 84| 84|Eltingville/Annad...|[6177331460644536...|
 Manhattan| 153| 153| Marble Hill|[6177331229858201...|
 Queens| 201| 201| Rockaway Park|[6177331357831659...|
 Queens| 129| 129| Jackson Heights|[6177331243222302...|
 Manhattan| 202| 202| Roosevelt Island|[6177331237777571...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

## 5. ‘Explode’ the polygon index dataframe, such that each polygon index becomes a row in a new dataframe.

In [0]:
exploded_indexed_neighbourhoods = (
  indexed_neighbourhoods
  .withColumn("ix", explode("ix_set"))
  .drop("ix_set")
)

exploded_indexed_neighbourhoods.show()

+--------+-----------+--------+--------------------+------------------+
 borough|location_id|objectid| zone| ix|
+--------+-----------+--------+--------------------+------------------+
Brooklyn| 255| 255|Williamsburg (Nor...|617733123827040255|
Brooklyn| 255| 255|Williamsburg (Nor...|617733124012638207|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123826253823|
Brooklyn| 255| 255|Williamsburg (Nor...|617733124013686783|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123827302399|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123788767231|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123789815807|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123789029375|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123826778111|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123788242943|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123825991679|
Brooklyn| 255| 255|Williamsburg (Nor...|617733124014211071|
Brooklyn| 255| 255|Williamsburg (Nor...|617733123818913791|
 Bronx| 147| 147| Longwood|617733123119775743|
 Bronx| 147| 147| Longwood|617733123112173567|
 Bronx| 147| 147| Longwood|617733123111649279|
 Bronx| 147| 147| Longwood|617733123110862847|
 Bronx| 147| 147| Longwood|617733123123707903|
 Bronx| 147| 147| Longwood|617733123111911423|
 Bronx| 147| 147| Longwood|617733123124756479|
+--------+-----------+--------+--------------------+------------------+
only showing top 20 rows

## 6. Join the new left- and right-hand dataframes directly on the index.

In [0]:
joined_df = (
  indexed_trips.alias("t")
  .join(exploded_indexed_neighbourhoods.alias("n"), on="ix", how="inner"))
joined_df.count()

Out[8]: 4921410

## Final notes
Mosaic provides support for Uber’s H3 spatial indexing library as a core part of the API, but we plan to add support for other index systems, including S2 and British National Grid in due course.